# 1 Import library

In [1]:
import findspark
findspark.init('/home/kate/spark-3.3.3-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession

In [2]:
import os
import fnmatch
from datetime import datetime
import json

In [3]:
import csv

In [4]:
pip show pyspark

Name: pyspark
Version: 3.3.3
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /home/kate/spark-3.3.3-bin-hadoop3/python
Requires: py4j
Required-by: delta-spark
Note: you may need to restart the kernel to use updated packages.


In [5]:
from delta.tables import DeltaTable
from delta import *

In [6]:
import warnings

warnings.filterwarnings("ignore")

# 2 Function

In [10]:
 #если нужен запуск до функции
#from delta import *

#builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
#    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

#spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [7]:
def create_mirror(directory, name_table, unique_key):
    mirror_path = "task_2.2/mirror"
    output_path = "task_2.2/mirr_md_account_d"
    log_file_path = "task_2.2/mirror/my_log.csv" 
    out = None
    mirror = None
    name_table_path = name_table + '*'
    name_table_change = None
    merge_condition = " AND ".join([f"del1.{key} = del2.{key}" for key in unique_key])
    
    #открываем спарк сессию
    builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    
    #рекурсивно перебираем файлы в директории
    for root, dirs, files in sorted(os.walk(directory)):
        for name in sorted(files):
            if fnmatch.fnmatch(name, name_table_path): #матч имени таблицы
                delta_file_path = os.path.join(root, name)
                time_start = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                delta_df = spark.read.csv(delta_file_path, header=True, sep=';', inferSchema=True)

                if not os.path.exists(log_file_path): #проверяем существование пути
                    delta_df.write.format("delta").mode("overwrite").save(mirror_path) #запись дата фрейма в формат дельта
                    mirror = DeltaTable.forPath(spark, mirror_path) #создаем дельта таблицу
                    time_end = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    out = spark.read.format("delta").load(mirror_path)
                    out.show()
                    name_table_change = log_delta_processed(time_start, time_end, mirror, delta_file_path, log_file_path)

                else:
                    if not check_table_name(log_file_path, delta_file_path): #проверяем наличие таблицы в логах
                        if mirror is None:
                            mirror = DeltaTable.forPath(spark, mirror_path)
                            #соединение таблиц по ключу
                            mirror.alias("del1").merge(  
                               source=delta_df.alias("del2"),
                               condition=merge_condition
                           ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
                            time_end = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                            out = spark.read.format("delta").load(mirror_path)
                            out.show()
                            name_table_change = log_delta_processed(time_start, time_end, mirror, delta_file_path, log_file_path)

                        else:
                            mirror.alias("del1").merge(
                               source=delta_df.alias("del2"),
                               condition=merge_condition
                           ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
                            time_end = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                            out = spark.read.format("delta").load(mirror_path)
                            out.show()
                            name_table_change = log_delta_processed(time_start, time_end, mirror, delta_file_path, log_file_path)


    if out is not None:
        out.write.csv(output_path, header=True,mode="overwrite") #запись в итоговый файл

    spark.stop()

def check_table_name(log_file_path, delta_file_path):
    if os.path.exists(log_file_path):
        with open(log_file_path, "r") as log_file:
            csvreader = csv.reader(log_file)
            for row in csvreader:
                if delta_file_path.split("/")[-1].split(".")[-2] in row:
                    return True
    return False

def log_delta_processed(time_start, time_end, mirror, delta_file_path, log_file_path):
    delta_id = delta_file_path.split("/")[-2]
    name_table_change = delta_file_path.split("/")[-1].split(".")[-2]
    log_message = [name_table_change, delta_id, time_start, time_end]
    header=["table_name", "delta_id", "time_start", "time_end"] 
    
    if not os.path.exists(log_file_path):
        with open(log_file_path, "w", newline='') as write_file:
            csv_writer = csv.writer(write_file)
            csv_writer.writerow(header)
            csv_writer.writerow(log_message)
    else:
        with open(log_file_path, "a", newline='') as write_file:
            csv_writer = csv.writer(write_file)
            csv_writer.writerow(log_message)

    return name_table_change

In [9]:
name_table = "md_account_d" #наименование таблицы
directory="task_2.2/data_deltas" #путь где хранятся дельты
unique_key=["ACCOUNT_RK"] #список полей, являющимся уникальным ключём
create_mirror(directory, name_table, unique_key)

23/09/18 14:15:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/18 14:15:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/09/18 14:15:37 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/09/18 14:15:37 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/09/18 14:15:37 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+---------+---------+----------------+
|DATA_ACTUAL_DATE|DATA_ACTUAL_END_DATE|ACCOUNT_RK|      ACCOUNT_NUMBER|CHAR_TYPE|CURRENCY_RK|CURRENCY_CODE|CLIENT_ID|BRANCH_ID|OPEN_IN_INTERNET|
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+---------+--------

In [33]:
import json